# Measuring the ATC Induction Profile of mNeon Green

In [46]:
import numpy as np
import pandas as pd
import bokeh.plotting
import glob
import bokeh.io
import skimage.io
import lac.viz
import lac.image
import tqdm
bokeh.io.output_notebook()

Loading BokehJS ...

This notebook performs a cursory analysis of the ATC induction profile of our newly-minted mNeonGreen tagged LacI dimers. We ultimately wish to find the appropriate induction profile that will permit observation of single puncta without a high background. This may result in only a few cells per field of view with appreciable fluorescent signal.


## Testing Segmentation

Muir (@muirjm) took a collection of images of cells with the mNeonGreen-LacI fusion expressed in a background which had a 5xOid array integrated at the *yffO* locus. He took images in both phase contrast and mNeon. We will first load the data and determine a segmentation scheme that seems appropriate. I have imported a few functions I used in the MscL project for segmentation that may prove useful.

In [5]:
# get a list of all of the files acquired at 1000 ms exposure
folders = glob.glob('../../../data/images/20200115*/*1000ms*')

# Choose the first entry and find the images (excluding metadata fro now). 
image_files = glob.glob(f'{folders[0]}/*.tif')

The images are saved as `ome.tif` files which are multi-dimensional files. There should be one phase contrast and one fluorescence image. Let's look at each

In [14]:
# Load the image
ome_image = skimage.io.imread(image_files[0])

# Display the two images. 
im1 = lac.viz.imshow(ome_image[0])
im1.title.text = "phase contrast"
im2 = lac.viz.imshow(ome_image[1])
im2.title.text = "mNeonGreen"
lay = bokeh.layouts.row(im1, im2)
bokeh.io.show(lay)

The phase contrast image is crisp enough that I may be able to use the contour segmentation method that I developed for the MscL project. Let's segment the phase image using this approach and see how well it works.


In [29]:
# Apply the contour segmentation method. 
phase_image = ome_image[0]
contours, labels = lac.image.contour_seg(phase_image, level=0.35, return_conts=True, 
                                        perim_bounds=(5, 1E5), area_bounds=(1, 100))

p = lac.viz.imshow(labels)
bokeh.io.show(p)

It looks like the contour segmentation works well with a `level=0.35`. The perimeter and area bounds can be improved but for right now I think it is okay. 

## Segmentation and quantification of all images

With the contour segmentation approach validated, we can now apply it to each image and extract the total fluoresence intensity. We will store all of the data in a tidy data frame where each row is an observed cell (cell used loosely -- more like an object)

In [83]:
# Instantiate the list to store the dataframes. 
dfs = []

# Get all of the image files from the directory.
image_files = glob.glob('../../../data/images/20200115*/**/*.tif')

# Drop those that have highpower. 
image_files = [i for i in image_files if 'highpower' not in i]

# Iterate through each image file. 
for i, im in enumerate(tqdm.tqdm(image_files)):
    # Parse the relevant information from the folder name. 
    fname = im.split('/')[-1]
    if 'auto' in fname:
        strain = 'auto'
        atc_ngml = 0 
    else:
        strain, conc, _, _, _, _ = fname.split('_')
        
        conc = conc.split('ngmL')[0]
        if 'p' in conc:
            atc_ngml = float(f"0.{conc.split('p')[-1]}")
        else:
            atc_ngml = float(conc)
    
    # Load and separate the images
    ome = skimage.io.imread(im)
    phase = ome[0]
    fluo = ome[1]

    # Segment the image
    seg = lac.image.contour_seg(phase, level=0.35, perim_bounds=(5, 1E5),
                                area_bounds=(1, 100))
    
    # Get the region properties from the fluorescence images. 
    props = skimage.measure.regionprops(seg, fluo)
    areas = np.array([p.area for p in props])
    mean_int = np.array([p.mean_intensity for p in props])
    total_int = mean_int * areas

    # if atc conc >= 1, correct the intensity by a factor of two
    if atc_ngml >= 1:
        mean_int *= 2
        total_int *= 2

    # Set up the dataframe. 
    df = pd.DataFrame(np.array([areas, mean_int, total_int]).T, columns=['area_pix', 'mean_int', 'total_int'])
    df['strain'] = strain
    df['exposure_ms'] = 1000
    df['atc_ngml'] = atc_ngml
    dfs.append(df)

# Concatenate into a single dataframe
intensity_data = pd.concat(dfs, sort=False)

# Save the data to disk
intensity_data.to_csv('output/20200115_atc_titration.csv', index=False)

0%|          | 0/50 [00:00<?, ?it/s]/Users/gchure/anaconda3/lib/python3.7/site-packages/skimage/measure/_regionprops.py:250: UserWarning: regionprops and image moments (including moments, normalized moments, central moments, and inertia tensor) of 2D images will change from xy coordinates to rc coordinates in version 0.16.
See https://scikit-image.org/docs/0.14.x/release_notes_and_installation.html#deprecations for details on how to avoid this message.
  warn(XY_TO_RC_DEPRECATION_MESSAGE)
/Users/gchure/anaconda3/lib/python3.7/site-packages/skimage/measure/_regionprops.py:260: UserWarning: regionprops and image moments (including moments, normalized moments, central moments, and inertia tensor) of 2D images will change from xy coordinates to rc coordinates in version 0.16.
See https://scikit-image.org/docs/0.14.x/release_notes_and_installation.html#deprecations for details on how to avoid this message.
  warn(XY_TO_RC_DEPRECATION_MESSAGE)
100%|██████████| 50/50 [01:07<00:00,  1.34s/it]


## Generating the induction curve
We can plot the cursory induction profile. I'll show the error as the SEM instead of STD.

In [86]:
# Perform the background subtraction
mean_auto = intensity_data[intensity_data['strain']=='auto']['mean_int'].mean()
intensity_data['total_int_sub'] = intensity_data['area_pix'].values * (intensity_data['mean_int'].values - mean_auto)

# Compute the aggregate properties on the bg subtracted signal
grouped = intensity_data[intensity_data['strain']=='yffO'].groupby(['atc_ngml'])['total_int_sub'].agg(('mean', 'sem')).reset_index()

# For ease of use, compute the bounds of the errorbars. 
grouped['lower'] = grouped['mean'].values - grouped['sem'].values 
grouped['upper'] = grouped['mean'].values + grouped['sem'].values

# Convert the dataframe to a CDS
source = ColumnDataSource(grouped)

# Generate the plot of mean intensity as a function of ATC concentration. 
p = bokeh.plotting.figure(width=600, height=400, x_axis_label='ATC [ng / mL]', y_axis_label='mean cell intensity [a.u.]', y_axis_type='log')

# Add the points, lines, and errors
p.circle(x='atc_ngml', y='mean', size=5, source=source, color='black')
p.line(x='atc_ngml', y='mean', line_width=1, source=source, color='black')
p.add_layout(Whisker(base='atc_ngml', lower='lower', upper='upper', source=source))
bokeh.io.show(p)

This is interesting and worth repeating. I have a feeling the 2 ngml point is a bit of an artefact and not representative (note it has larger error as well). Since we are operating in the neongreen world, it would probably be worthwhile to see what the overall fluorescence of trace ATC in the medium is. However, I think this is a good representative titration curve that suggests we need to operate between 0 and 0.5 ng/mL of ATC. 